In [64]:
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np


from utils import qubo_dict_to_matrix
from make_qubo import build_qubo
from evaluate import evaluate_solution
from utils import bitstring_to_assignment, generate_lock_types
from data_processing import generate_ship_data

num_vars = 9

ship_data = generate_ship_data(3)
L = ship_data["Length (m)"].to_numpy()
B = ship_data["Benefit"].to_numpy()
lock_types = generate_lock_types(3)

qubo = build_qubo(B, L, lock_types)
Q_mat = qubo_dict_to_matrix(qubo)

print(Q_mat)

# Build the QuadraticProgram
qp = QuadraticProgram()
for i in range(num_vars):
    qp.binary_var(name=f'x_{i}')

linear = {}
quadratic = {}
for i in range(num_vars):
    linear[f'x_{i}'] = Q_mat[i, i]
    for j in range(i + 1, num_vars):
        quadratic[(f'x_{i}', f'x_{j}')] = Q_mat[i, j]
        
qp.minimize(linear=linear, quadratic=quadratic)

op, offset = qp.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

[[ 95. 110. 110. 100.   0.   0. 100.   0.   0.]
 [  0.  95. 110.   0. 100.   0.   0. 100.   0.]
 [  0.   0. -55.   0.   0.  55.   0.   0.  55.]
 [  0.   0.   0. -55. 110. 110. 100.   0.   0.]
 [  0.   0.   0.   0. -55. 110.   0. 100.   0.]
 [  0.   0.   0.   0.   0. -55.   0.   0. 105.]
 [  0.   0.   0.   0.   0.   0. -55. 110. 110.]
 [  0.   0.   0.   0.   0.   0.   0. -55. 110.]
 [  0.   0.   0.   0.   0.   0.   0.   0. -55.]]
offset: 353.75
operator:
SparsePauliOp(['IIIIIIIIZ', 'IIIIIIIZI', 'IIIIIIZII', 'IIIIIZIII', 'IIIIZIIII', 'IIIZIIIII', 'IIZIIIIII', 'IZIIIIIII', 'ZIIIIIIII', 'IIIIIIIZZ', 'IIIIIIZIZ', 'IIIIIZIIZ', 'IIZIIIIIZ', 'IIIIIIZZI', 'IIIIZIIZI', 'IZIIIIIZI', 'IIIZIIZII', 'ZIIIIIZII', 'IIIIZZIII', 'IIIZIZIII', 'IIZIIZIII', 'IIIZZIIII', 'IZIIZIIII', 'ZIIZIIIII', 'IZZIIIIII', 'ZIZIIIIII', 'ZZIIIIIII'],
              coeffs=[-152.5 +0.j, -152.5 +0.j,  -55.  +0.j,  -77.5 +0.j,  -77.5 +0.j,
  -67.5 +0.j,  -77.5 +0.j,  -77.5 +0.j,  -67.5 +0.j,   27.5 +0.j,
   27.5 +0.j,   25.  +

In [65]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.prettyprint())

Problem name: 

Minimize
  110*x0*x1 + 110*x0*x2 + 100*x0*x3 + 100*x0*x6 + 110*x1*x2 + 100*x1*x4
  + 100*x1*x7 + 55*x2*x5 + 55*x2*x8 + 110*x3*x4 + 110*x3*x5 + 100*x3*x6
  + 110*x4*x5 + 100*x4*x7 + 105*x5*x8 + 110*x6*x7 + 110*x6*x8 + 110*x7*x8
  + 95*x0 + 95*x1 - 55*x2 - 55*x3 - 55*x4 - 55*x5 - 55*x6 - 55*x7 - 55*x8

Subject to
  No constraints

  Binary variables (9)
    x0 x1 x2 x3 x4 x5 x6 x7 x8



In [66]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0,0])
exact_mes = NumPyMinimumEigensolver()

/tmp/ipykernel_14925/3227849292.py:2: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0,0])


In [67]:
print(np.zeros(num_vars))

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [68]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [69]:
exact_result = exact.solve(qp)
print(exact_result.prettyprint())

objective function value: -165.0
variable values: x0=0.0, x1=0.0, x2=1.0, x3=1.0, x4=0.0, x5=0.0, x6=0.0, x7=1.0, x8=0.0
status: SUCCESS


In [70]:
qaoa_result = qaoa.solve(qp)
print(qaoa_result.prettyprint())

objective function value: -165.0
variable values: x0=0.0, x1=0.0, x2=1.0, x3=0.0, x4=1.0, x5=0.0, x6=1.0, x7=0.0, x8=0.0
status: SUCCESS


In [71]:
print("variable order:", [var.name for var in qaoa_result.variables])
for s in qaoa_result.samples:
    print(s)

variable order: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']
SolutionSample(x=array([0., 0., 1., 0., 1., 0., 1., 0., 0.]), fval=-165.0, probability=0.001953125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 1., 1., 0., 0., 0., 1., 0.]), fval=-165.0, probability=0.001953125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 1., 1., 0., 0., 0., 0., 0.]), fval=-110.0, probability=0.001953125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 1., 0., 1., 0., 0., 0., 0.]), fval=-110.0, probability=0.001953125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 1., 0., 0., 0., 1., 0., 0.]), fval=-110.0, probability=0.001953125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 0., 0., 1., 0., 1., 0., 0.]), fval=-110.0, probability=0.001953125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 0., 0., 0., 1., 1., 0

In [72]:
print(qaoa_result.samples[0].x)

[0. 0. 1. 0. 1. 0. 1. 0. 0.]


In [73]:
def convert_array_to_bitstring(x_array):
    """
    Converts a numpy array of 0s and 1s into a bitstring.
    Example: array([0., 1., 0., 1.]) -> '0101'
    """
    return ''.join(map(str, map(int, x_array)))

In [80]:
assignment = convert_array_to_bitstring(qaoa_result.samples[0].x)

In [83]:
print(assignment)

001010100


In [82]:
from evaluate import evaluate_solution

# Evaluate only the top 20% assignments.
evaluated_solutions = {
    evaluate_solution(assignment, B, L, lock_types)
}

# Print out the sorted top 10% bitstrings with their counts, assignments, and evaluations.
for bit in evaluated_solutions:
    print(f"Bitstring: {bit}")
    print(f"  Assignment: {assignment[bit]}")
    print(f"  Evaluation: {evaluated_solutions[bit]}")
    print("-" * 40)


TypeError: unsupported operand type(s) for +: 'int' and 'str'